In [181]:
import pandas
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta, MO, SU

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql import types as t
from pyspark.sql.window import Window as w

In [182]:
spark = SparkSession.builder\
    .master("local[*]")\
    .appName("SparkDemo")\
    .getOrCreate()

## Explore data

In [183]:
video_categories_df = spark.read.format("json") \
    .option("multiline","true") \
    .load(f"./data/GB_category_id.json")

In [184]:
video_categories_df.printSchema()

root
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- assignable: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |-- kind: string (nullable = true)


In [185]:
video_categories_df = (
    video_categories_df
        .withColumn("categories", F.explode(F.arrays_zip("items.id", "items.snippet.title")))
        .select(
            col("categories")["id"].alias("id"),
            col("categories")["title"].alias("title"),
        )
)
video_categories_df.limit(5).toPandas().head()

,id,title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [186]:
videos_df = spark.read.format("csv") \
    .option("multiline", True) \
    .option("sep", ",") \
    .option("header", True) \
    .load(f"./data/GBvideos.csv")

videos_df.limit(10).toPandas().head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"""christmas""|""john lewis christmas""|""john lewis...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...
1,3s1rvMFUweQ,17.14.11,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,24,2017-11-12T06:24:44.000Z,"""SNL""|""Saturday Night Live""|""SNL Season 43""|""E...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"""Eminem""|""Walk""|""On""|""Water""|""Aftermath/Shady/...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,17,2017-11-13T02:30:38.000Z,"""Salford City FC""|""Salford City""|""Salford""|""Cl...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,17.14.11,Dashcam captures truck's near miss with child ...,Cute Girl Videos,25,2017-11-13T01:45:13.000Z,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...


### Check nulls in each column in dataframes

In [187]:
video_categories_df.select([F.count(F.when(F.isnan(c), c)).alias(c) for c in video_categories_df.columns]).toPandas()

,id,title
0,0,0


In [188]:
videos_df.select([F.count(F.when(F.isnan(c), c)).alias(c) for c in videos_df.columns]).toPandas()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [189]:
videos_df.select(['trending_date']).distinct().head(10)

[Row(trending_date='18.08.05'),
 Row(trending_date='17.20.11'),
 Row(trending_date='17.09.12'),
 Row(trending_date='18.14.02'),
 Row(trending_date='18.20.03'),
 Row(trending_date='18.04.05'),
 Row(trending_date='18.21.02'),
 Row(trending_date='18.30.04'),
 Row(trending_date='18.06.05'),
 Row(trending_date='18.11.02')]

In [190]:
def df_print(df, n_rows=5):
    return df.limit(n_rows).toPandas().head(n_rows)


def convert_df_to_column_array(old_df, from_col, to_col):
    new_df = (
        old_df
            .agg(F.collect_list(col(from_col)).alias(to_col))
            .select([to_col])
    )
    return new_df

## Query 1

**Description:** Find Top 10 videos that were amongst the trending videos for the highest
number of days (it doesn't need to be a consecutive period of time).
You should also include information about different metrics for each day
the video was trending.

In [191]:
# Create initial dataframe for the query
df = videos_df.select(
    col('video_id'),
    col("title"),
    col("description"),
    F.struct(
        F.to_date(F.from_unixtime(F.unix_timestamp(col("trending_date"), "yy.dd.MM"))).alias("date"), 
        col("likes").cast(t.LongType()).alias("likes"), 
        col("dislikes").cast(t.LongType()).alias("dislikes"),
        col("views").cast(t.LongType()).alias("views")
    ).alias('trending_day')
)

# Get top 10 most trending video ids
most_trending_df = df.groupBy(df.video_id)\
                        .agg(F.count('video_id').alias('num_trending_days'))\
                        .sort(col('num_trending_days').desc()).limit(10)
df_print(most_trending_df, 10)

,video_id,num_trending_days
0,BhIEIO0vaBE,38
1,2z3EUY1aXdY,38
2,NooW_RbfdWI,38
3,Il-an3K9pjg,38
4,u_C4onVrr8U,38
5,Q17z9a03YaM,37
6,5GHXEGz3PJg,37
7,dzxFdtWmjto,37
8,tsp7IOr7Q9A,37
9,ABLsSWE06q4,37


In [192]:
# Get title, description and trending_day for each video from top 10
detailed_most_trending_df = most_trending_df.alias('df1').join(
    df.alias('df2'), col('df1.video_id') == col('df2.video_id'), 'inner')\
        .select(
            col('df1.video_id').alias('video_id'),
            col('num_trending_days'),
            col('title'),
            col('description'),
            col('trending_day')
        )
df_print(detailed_most_trending_df)

,video_id,num_trending_days,title,description,trending_day
0,BhIEIO0vaBE,38,To Our Daughter,Directed by Tyler Ross @wttyler\nMusic by Jaco...,"(2018-02-05, 0, 0, 20921796)"
1,NooW_RbfdWI,38,Jurassic World: Fallen Kingdom - Official Trai...,Jurassic World: Fallen Kingdom \nIn Theaters J...,"(2018-02-05, 61833, 1416, 1999326)"
2,2z3EUY1aXdY,38,Justin Timberlake’s FULL Pepsi Super Bowl LII ...,Justin Timberlake breaks it down during the Pe...,"(2018-02-05, 50251, 15239, 2027569)"
3,BhIEIO0vaBE,38,To Our Daughter,Directed by Tyler Ross @wttyler\nMusic by Jaco...,"(2018-02-06, 0, 0, 35832484)"
4,2z3EUY1aXdY,38,Justin Timberlake’s FULL Pepsi Super Bowl LII ...,Justin Timberlake breaks it down during the Pe...,"(2018-02-06, 123302, 39422, 8313413)"


In [193]:
# Find the latest day video statistics
window = w.partitionBy("video_id").orderBy(col("trending_day.date").desc())
latest_day_info_df = detailed_most_trending_df.withColumn("rank", F.row_number().over(window)) \
                        .where(col("rank") == 1) \
                        .select(
                            col("video_id"), 
                            col("title"), 
                            col("description"), 
                            col("trending_day.likes").alias("latest_likes"), 
                            col("trending_day.dislikes").alias("latest_dislikes"),
                            col("trending_day.views").alias("latest_views")
                        )
df_print(latest_day_info_df, 15)

,video_id,title,description,latest_likes,latest_dislikes,latest_views
0,5GHXEGz3PJg,Florence + The Machine - Hunger,HungerDirected by AG RojasProduced by Park Pic...,171071,3972,12293782
1,dARAN1z2KqY,"Royce da 5'9 - Caterpillar ft. Eminem, King Green",Music video by Royce 5'9 and Eminem performing...,336484,10827,17172722
2,ABLsSWE06q4,"Whethan, Dua Lipa - High (Audio)",High (Official Audio)\nSong available on the F...,108466,1525,7285185
3,2z3EUY1aXdY,Justin Timberlake’s FULL Pepsi Super Bowl LII ...,Justin Timberlake breaks it down during the Pe...,169320,54005,14251760
4,dzxFdtWmjto,VENOM - Official Teaser Trailer (HD),Watch the #Venom teaser trailer now. 10.5.18.\...,219113,25700,16416756
5,4uTNVumfm84,Maroon 5 - Wait,RED PILL BLUES” is out now.\nhttp://smarturl.i...,633772,16442,46205645
6,NooW_RbfdWI,Jurassic World: Fallen Kingdom - Official Trai...,Jurassic World: Fallen Kingdom \nIn Theaters J...,275268,7806,24293173
7,Il-an3K9pjg,Anne-Marie - 2002 [Official Video],Get 2002 by Anne-Marie HERE ▶ http://ad.gt/200...,394830,8892,29641412
8,BhIEIO0vaBE,To Our Daughter,Directed by Tyler Ross @wttyler\nMusic by Jaco...,0,0,62338362
9,u_C4onVrr8U,Miguel - Come Through and Chill (Official Vide...,“Come Through and Chill” ft. J. Cole out now! ...,152765,3266,9657370


In [194]:
# Get all trending days for top 10 videos
trending_days_df = latest_day_info_df.alias('df1')\
                            .join(df.alias('df2'),
                                    col('df1.video_id') == col('df2.video_id'), how='inner')\
                            .select(
                                col('df1.video_id').alias('video_id'),
                                col('trending_day')
                            )
trending_days_df = trending_days_df.groupBy("video_id")\
                                    .agg(F.collect_list(col("trending_day")).alias("trending_days"))
df_print(trending_days_df, 20)

,video_id,trending_days
0,5GHXEGz3PJg,"[(2018-06-09, 171071, 3972, 12293782), (2018-0..."
1,dARAN1z2KqY,"[(2018-06-09, 336484, 10827, 17172722), (2018-..."
2,ABLsSWE06q4,"[(2018-03-17, 108466, 1525, 7285185), (2018-03..."
3,2z3EUY1aXdY,"[(2018-03-14, 169320, 54005, 14251760), (2018-..."
4,dzxFdtWmjto,"[(2018-03-17, 219113, 25700, 16416756), (2018-..."
5,4uTNVumfm84,"[(2018-03-17, 633772, 16442, 46205645), (2018-..."
6,NooW_RbfdWI,"[(2018-03-14, 275268, 7806, 24293173), (2018-0..."
7,Il-an3K9pjg,"[(2018-06-14, 394830, 8892, 29641412), (2018-0..."
8,BhIEIO0vaBE,"[(2018-03-14, 0, 0, 62338362), (2018-03-13, 0,..."
9,u_C4onVrr8U,"[(2018-06-01, 152765, 3266, 9657370), (2018-05..."


In [195]:
# Join and get final results
final_df = latest_day_info_df.alias('df1')\
                        .join(trending_days_df.alias('df2'),
                              col('df1.video_id') == col('df2.video_id'))\
                        .select(
                            col('df1.video_id').alias('video_id'),
                            col('title'),
                            col('description'),
                            col('latest_views'),
                            col('latest_likes'),
                            col('latest_dislikes'),
                            col('trending_days')
                        )
df_print(final_df, 20)

,video_id,title,description,latest_views,latest_likes,latest_dislikes,trending_days
0,5GHXEGz3PJg,Florence + The Machine - Hunger,HungerDirected by AG RojasProduced by Park Pic...,12293782,171071,3972,"[(2018-06-09, 171071, 3972, 12293782), (2018-0..."
1,dARAN1z2KqY,"Royce da 5'9 - Caterpillar ft. Eminem, King Green",Music video by Royce 5'9 and Eminem performing...,17172722,336484,10827,"[(2018-06-09, 336484, 10827, 17172722), (2018-..."
2,ABLsSWE06q4,"Whethan, Dua Lipa - High (Audio)",High (Official Audio)\nSong available on the F...,7285185,108466,1525,"[(2018-03-17, 108466, 1525, 7285185), (2018-03..."
3,2z3EUY1aXdY,Justin Timberlake’s FULL Pepsi Super Bowl LII ...,Justin Timberlake breaks it down during the Pe...,14251760,169320,54005,"[(2018-03-14, 169320, 54005, 14251760), (2018-..."
4,dzxFdtWmjto,VENOM - Official Teaser Trailer (HD),Watch the #Venom teaser trailer now. 10.5.18.\...,16416756,219113,25700,"[(2018-03-17, 219113, 25700, 16416756), (2018-..."
5,4uTNVumfm84,Maroon 5 - Wait,RED PILL BLUES” is out now.\nhttp://smarturl.i...,46205645,633772,16442,"[(2018-03-17, 633772, 16442, 46205645), (2018-..."
6,NooW_RbfdWI,Jurassic World: Fallen Kingdom - Official Trai...,Jurassic World: Fallen Kingdom \nIn Theaters J...,24293173,275268,7806,"[(2018-03-14, 275268, 7806, 24293173), (2018-0..."
7,Il-an3K9pjg,Anne-Marie - 2002 [Official Video],Get 2002 by Anne-Marie HERE ▶ http://ad.gt/200...,29641412,394830,8892,"[(2018-06-14, 394830, 8892, 29641412), (2018-0..."
8,BhIEIO0vaBE,To Our Daughter,Directed by Tyler Ross @wttyler\nMusic by Jaco...,62338362,0,0,"[(2018-03-14, 0, 0, 62338362), (2018-03-13, 0,..."
9,u_C4onVrr8U,Miguel - Come Through and Chill (Official Vide...,“Come Through and Chill” ft. J. Cole out now! ...,9657370,152765,3266,"[(2018-06-01, 152765, 3266, 9657370), (2018-05..."


In [196]:
top_videos_df = final_df.select(
                            F.struct(
                                col("video_id"),
                                col('title'),
                                col('description'),
                                col('latest_views'),
                                col('latest_likes'),
                                col('latest_dislikes'),
                                col('trending_days')
                            ).alias("video")
                        )
df_print(top_videos_df, 10)

,video
0,"(5GHXEGz3PJg, Florence + The Machine - Hunger,..."
1,"(dARAN1z2KqY, Royce da 5'9 - Caterpillar ft. E..."
2,"(ABLsSWE06q4, Whethan, Dua Lipa - High (Audio)..."
3,"(2z3EUY1aXdY, Justin Timberlake’s FULL Pepsi S..."
4,"(dzxFdtWmjto, VENOM - Official Teaser Trailer ..."
5,"(4uTNVumfm84, Maroon 5 - Wait, RED PILL BLUES”..."
6,"(NooW_RbfdWI, Jurassic World: Fallen Kingdom -..."
7,"(Il-an3K9pjg, Anne-Marie - 2002 [Official Vide..."
8,"(BhIEIO0vaBE, To Our Daughter, Directed by Tyl..."
9,"(u_C4onVrr8U, Miguel - Come Through and Chill ..."


In [197]:
q1_videos_df = convert_df_to_column_array(top_videos_df, from_col="video", to_col="videos")
df_print(q1_videos_df)

,videos
0,"[(5GHXEGz3PJg, Florence + The Machine - Hunger..."


## Query 2

**Description:** Find what was the most popular category for each week (7 days slices).
Popularity is decided based on the total number of views for videos of
this category. Note, to calculate it you can’t just sum up the number of views.
If a particular video appeared only once during the given period, it shouldn’t be
counted. Only if it appeared more than once you should count the number of new
views. For example, if video A appeared on day 1 with 100 views, then on day 4
with 250 views and again on day 6 with 400 views, you should count it as 400 - 100 = 300.
For our purpose, it will mean that this particular video was watched 300 times
in the given time period.

In [198]:
week_dates_schema = t.StructType([
    t.StructField("start_date", t.DateType()),
    t.StructField("end_date", t.DateType())
])


@F.udf(week_dates_schema)
def get_week_dates_udf(trending_date):
    trending_datetime = datetime.strptime(str(trending_date), "%Y-%m-%d")
    start_date = trending_datetime + relativedelta(weekday=MO(-1))
    end_date = trending_datetime + relativedelta(weekday=SU(1))
    return start_date, end_date


In [199]:
# Split video ids on 7-days chunks
df = videos_df.select(
        col("video_id"),
        F.to_date(F.from_unixtime(F.unix_timestamp(col("trending_date"), "yy.dd.MM"))).alias("trending_date"), 
        col("views").cast(t.LongType()).alias("views"), 
        col("category_id")
    )

chunks_df = df.withColumn("week_dates", get_week_dates_udf(col("trending_date"))) \
                .select(
                    col("video_id"),
                    col("views"),
                    col("category_id"),
                    col("trending_date"),
                    col("week_dates.start_date").alias("start_date"),
                    col("week_dates.end_date").alias("end_date"))

In [200]:
chunks_df.show()

+-----------+--------+-----------+-------------+----------+----------+
|   video_id|   views|category_id|trending_date|start_date|  end_date|
+-----------+--------+-----------+-------------+----------+----------+
|Jw1Y-zhQURU| 7224515|         26|   2017-11-14|2017-11-13|2017-11-19|
|3s1rvMFUweQ| 1053632|         24|   2017-11-14|2017-11-13|2017-11-19|
|n1WpP7iowLc|17158579|         10|   2017-11-14|2017-11-13|2017-11-19|
|PUTEiSjKwJU|   27833|         17|   2017-11-14|2017-11-13|2017-11-19|
|rHwDegptbI4|    9815|         25|   2017-11-14|2017-11-13|2017-11-19|
|AumaWl0TNBo| 1182775|         24|   2017-11-14|2017-11-13|2017-11-19|
|2Vv-BfVoq4g|33523622|         10|   2017-11-14|2017-11-13|2017-11-19|
|-N5eucPMTTc| 1164201|         22|   2017-11-14|2017-11-13|2017-11-19|
|LMCuKltaY3M|  154494|         10|   2017-11-14|2017-11-13|2017-11-19|
|9t9u_yPEidY| 9548677|         10|   2017-11-14|2017-11-13|2017-11-19|
|ONQ-fAp5X64|  306724|         17|   2017-11-14|2017-11-13|2017-11-19|
|Dlwf5

In [201]:
# Count number of video appearance during this 7 days. 
# And filter video ids, which appeared more than once in its period chunk
weekly_video_views_df = (
    chunks_df
        .groupBy("video_id", "category_id", "start_date", "end_date")
        .agg(
            F.count("video_id").alias("video_count"), 
            F.collect_list(col("views")).alias("views_lst")
        )
        .where(col("video_count") >= 2)
        .select(
            col("video_id"),
            col("start_date"),
            col("end_date"),
            col("category_id"),
            col("views_lst")
        )
)

In [202]:
# Get views_delta for videos, which relates to the same period chunk and category, sum it 
# and find the mpst popular category for each week
window = w.partitionBy("start_date", "end_date").orderBy(col("total_views").desc())
weekly_most_popular_categories_df = (
    weekly_video_views_df
        .withColumn("views_delta", F.array_max(col("views_lst")) - F.array_min(col("views_lst")))
        .groupBy("start_date", "end_date", "category_id")
        .agg(
            F.count("video_id").alias("number_of_videos"), 
            F.sum(col("views_delta")).alias("total_views"),
            F.collect_list(col("video_id")).alias("video_ids_lst")
        )
        .withColumn("rank", F.row_number().over(window))
        .where(col("rank") == 1)
)
weekly_most_popular_categories_df.toPandas().head(10)

,start_date,end_date,category_id,number_of_videos,total_views,video_ids_lst,rank
0,2018-01-15,2018-01-21,10,88,199061812,"[zQYarS9LYgk, eACohWVwTOc, m-Q0Ng61bT4, M4vnmg...",1
1,2018-03-19,2018-03-25,10,100,395632626,"[gegW-vR_UHo, y0PMqR7IbAQ, elwTgpHlty0, gNTIOZ...",1
2,2018-02-05,2018-02-11,10,75,224470117,"[vAh7jTAkCKc, NsJkUfrqLzY, 4uTNVumfm84, 900X9f...",1
3,2017-11-27,2017-12-03,10,69,113428687,"[SA7AIQw-7Ms, UceaB4D0jpo, qgGooEyeNNc, o9zRQi...",1
4,2018-01-22,2018-01-28,10,90,159436718,"[ty-AFKiiTRA, JWH5KE1atAg, Xo8FPXlI3CU, 8BQqX3...",1
5,2018-02-26,2018-03-04,10,84,350338453,"[9TRjE7i0ERY, kton-qrKNLc, tEnCoocmPQM, JQbjS0...",1
6,2018-05-28,2018-06-03,10,86,290276954,"[Il-an3K9pjg, igT1QKKm2GE, mQ055hHdxbE, YynKel...",1
7,2018-05-14,2018-05-20,10,90,361439197,"[uovntV3ZMDc, 71Es-8FfATo, UvDJ-ZdHiSg, 5GHXEG...",1
8,2018-03-05,2018-03-11,10,88,391858363,"[aMGQgC5yV-o, LjUXm0Zy_dk, jzD_yyEcp0M, L0xKz5...",1
9,2018-04-09,2018-04-15,10,86,50262613,"[H95362B0YOg, aZUopuhqwCM, 7snvaAmSHo8, fGqdIP...",1


In [203]:
# Join to get category titles
final_df = (
    weekly_most_popular_categories_df.alias("df1")
        .join(video_categories_df.alias("df2"), col("df1.category_id") == col("df2.id"), how="left")
        .sort(col("df1.start_date").desc())
        .select(
            col("df1.start_date"),
            col("df1.end_date"),
            col("df1.category_id"),
            col("df2.title").alias("category_name"),
            col("df1.number_of_videos"),
            col("df1.total_views"),
            col("df1.video_ids_lst").alias("video_ids")
        )
)
final_df.toPandas().head(50)

,start_date,end_date,category_id,category_name,number_of_videos,total_views,video_ids
0,2018-06-11,2018-06-17,10,Music,79,83560849,"[8JnfIa84TnU, 2MiQonPvlVM, BspHjvU11y4, 3KmIMu..."
1,2018-06-04,2018-06-10,10,Music,95,280816732,"[Ibp6KqGRVbY, X1VzzNbfPaM, Kbj2Zss-5GY, Wpm07-..."
2,2018-05-28,2018-06-03,10,Music,86,290276954,"[Il-an3K9pjg, igT1QKKm2GE, mQ055hHdxbE, YynKel..."
3,2018-05-21,2018-05-27,10,Music,85,345903363,"[Ck4xHocysLw, 3rkXFyt7EXw, Ibp6KqGRVbY, G3Sl2v..."
4,2018-05-14,2018-05-20,10,Music,90,361439197,"[uovntV3ZMDc, 71Es-8FfATo, UvDJ-ZdHiSg, 5GHXEG..."
5,2018-05-07,2018-05-13,10,Music,102,533427464,"[l884wKofd54, BNzc6hG3yN4, 9jI-z9QN6g8, Pgd_Tg..."
6,2018-04-30,2018-05-06,10,Music,103,356479804,"[KQRq9FZqNj0, iPf_ifCKftc, e1YqueG2gtQ, ufynqs..."
7,2018-04-23,2018-04-29,10,Music,110,351787105,"[Pgd_Tgi-pbQ, CPy0-J6pxKI, h208HbsyTzs, J75eny..."
8,2018-04-16,2018-04-22,10,Music,98,317698876,"[vUTI4bPdlgE, PVbQrvlB_gw, O02MnrL_wrc, Ks6I2r..."
9,2018-04-09,2018-04-15,10,Music,86,50262613,"[H95362B0YOg, aZUopuhqwCM, 7snvaAmSHo8, fGqdIP..."


In [204]:
final_df = final_df.select(
                            F.struct(
                                col("start_date"),
                                col("end_date"),
                                col("category_id"),
                                col("category_name"),
                                col("number_of_videos"),
                                col("total_views"),
                                col("video_ids")
                            ).alias("week")
                        )
final_df.toPandas().head(10)

,week
0,"(2018-06-11, 2018-06-17, 10, Music, 79, 835608..."
1,"(2018-06-04, 2018-06-10, 10, Music, 95, 280816..."
2,"(2018-05-28, 2018-06-03, 10, Music, 86, 290276..."
3,"(2018-05-21, 2018-05-27, 10, Music, 85, 345903..."
4,"(2018-05-14, 2018-05-20, 10, Music, 90, 361439..."
5,"(2018-05-07, 2018-05-13, 10, Music, 102, 53342..."
6,"(2018-04-30, 2018-05-06, 10, Music, 103, 35647..."
7,"(2018-04-23, 2018-04-29, 10, Music, 110, 35178..."
8,"(2018-04-16, 2018-04-22, 10, Music, 98, 317698..."
9,"(2018-04-09, 2018-04-15, 10, Music, 86, 502626..."


In [205]:
q2_weeks_df = convert_df_to_column_array(final_df, from_col="week", to_col="weeks")
df_print(q2_weeks_df)

,weeks
0,"[(2018-06-11, 2018-06-17, 10, Music, 79, 83560..."


## Query 3

**Description:** What were the 10 most used tags amongst trending videos for each 30days time period?
Note, if during the specified period the same video appears multiple times,
you should count tags related to that video only once.

In [206]:
# Create 30-days windows and filter periods by distinct video_ids
df = (
    videos_df
        .withColumn('tags', F.regexp_replace(col("tags"), '\"', ''))
        .withColumn('trending_date',
                    F.to_date(F.from_unixtime(F.unix_timestamp(col("trending_date"), "yy.dd.MM"))) )
        .withColumn("30_days_period", F.window("trending_date", "30 days"))
        .select(
            col('video_id'),
            col('tags'),
            col('30_days_period.start').alias('start_date'),
            col('30_days_period.end').alias('end_date')
        ).distinct()
)
df.toPandas().head(10)

,video_id,tags,start_date,end_date
0,qqs3fxfmWr4,pencils|how it's made|science channel|design|f...,2017-10-21 03:00:00,2017-11-20 02:00:00
1,r81egea0QPo,Bohemian Rhapsody Live AID|freddie mercury fir...,2017-11-20 02:00:00,2017-12-20 02:00:00
2,AenmqoozqoA,Dirty Sexy Money|David Guetta|Guetta|Afrojack|...,2017-11-20 02:00:00,2017-12-20 02:00:00
3,muAPJiMho2Y,Z800|Z800E|KAWASAKI|PARIS|GOPRO|CAM|MOTO|PARIS...,2017-12-20 02:00:00,2018-01-19 02:00:00
4,LPBPma9YJnE,Daisy Ridley Says Rey is MORE Powerful than AN...,2017-12-20 02:00:00,2018-01-19 02:00:00
5,2UmRhLA3Qzk,Nintendo|Labo|Nintendo Labo|Workshop|Toy-Con|M...,2017-12-20 02:00:00,2018-01-19 02:00:00
6,Zukh-0csKPc,[none],2018-01-19 02:00:00,2018-02-18 02:00:00
7,JVDyowAzBrU,#pabllo|#vittar|#pabllo vittar|#pv|diplo|entao...,2018-01-19 02:00:00,2018-02-18 02:00:00
8,Zt803kKQ41k,tami roman|basketball wives|vh1|nba|basketball...,2018-01-19 02:00:00,2018-02-18 02:00:00
9,V0nfRcc4PJ0,Black Panther (Marvel Comics)|Marvel Studios|B...,2018-01-19 02:00:00,2018-02-18 02:00:00


In [207]:
# Check if window function worked correctly
df.select(['start_date', 'end_date']).distinct().sort(col('start_date')).show()

+-------------------+-------------------+
|         start_date|           end_date|
+-------------------+-------------------+
|2017-10-21 03:00:00|2017-11-20 02:00:00|
|2017-11-20 02:00:00|2017-12-20 02:00:00|
|2017-12-20 02:00:00|2018-01-19 02:00:00|
|2018-01-19 02:00:00|2018-02-18 02:00:00|
|2018-02-18 02:00:00|2018-03-20 02:00:00|
|2018-03-20 02:00:00|2018-04-19 03:00:00|
|2018-04-19 03:00:00|2018-05-19 03:00:00|
|2018-05-19 03:00:00|2018-06-18 03:00:00|
+-------------------+-------------------+


In [209]:
# Group by 30-days periods and count number of tags and video_ids for each period 
period_df = (
      df
        .withColumn('tags_array', F.split(col('tags'), "\|"))
        .withColumn('tag', F.explode(col('tags_array')))
        .groupBy('tag', 'start_date', 'end_date')
        .agg(
            F.count(col('video_id')).alias('number_of_videos'), 
            F.collect_list(col('video_id')).alias('video_ids')
        )
        .select(['tag', 'number_of_videos', 'video_ids', 'start_date', 'end_date'])
)
df_print(period_df, 10)

,tag,number_of_videos,video_ids,start_date,end_date
0,#chucklorre,1,[JXZG_Qcm6Nw],2018-05-19 03:00:00,2018-06-18 03:00:00
1,2017 roadster,1,[5n9xafjynJA],2017-10-21 03:00:00,2017-11-20 02:00:00
2,2019,1,[aXWfL-1ieuE],2017-10-21 03:00:00,2017-11-20 02:00:00
3,2CELLOS,1,[ozkqm2ifMw8],2017-10-21 03:00:00,2017-11-20 02:00:00
4,A Good Night,1,[KQRq9FZqNj0],2018-03-20 02:00:00,2018-04-19 03:00:00
5,ASMR,1,[TYVD7XzW55I],2018-03-20 02:00:00,2018-04-19 03:00:00
6,Adam Rippon,1,[qOioqgPyCwE],2018-04-19 03:00:00,2018-05-19 03:00:00
7,Album,1,[Uz-uEW_Rur4],2017-11-20 02:00:00,2017-12-20 02:00:00
8,Anchorman,1,[zbGSbZXzvqw],2018-04-19 03:00:00,2018-05-19 03:00:00
9,BMX STREET,1,[8mFUhpKz01s],2017-11-20 02:00:00,2017-12-20 02:00:00


In [210]:
# Sort number_of_videos in descending order. Take top 10 most used tags
window =  w.partitionBy("start_date", "end_date").orderBy(col("number_of_videos").desc())
top_tags_df = (
    period_df
            .withColumn("rank", F.row_number().over(window))
            .where(col("rank") <= 10)
            .select(
                col("start_date"),
                col("end_date"),
                F.struct(
                    col("tag"),
                    col("number_of_videos"),
                    col("video_ids")
                ).alias("tag_stat")
            )
    )
df_print(top_tags_df, 10)

,start_date,end_date,tag_stat
0,2018-03-20 02:00:00,2018-04-19 03:00:00,"(comedy, 35, [H4NjcO2GAMU, BYG1qf3XJNM, kEU5V4..."
1,2018-03-20 02:00:00,2018-04-19 03:00:00,"(funny, 34, [m6qubM54PtE, fkBYn3QkgpA, HfIgfy0..."
2,2018-03-20 02:00:00,2018-04-19 03:00:00,"(music, 33, [HfIgfy0HJHU, DfxhWPvhFCQ, 4hrrK3r..."
3,2018-03-20 02:00:00,2018-04-19 03:00:00,"([none], 30, [n0pMQHNjtxc, jnQ4V-wajLY, I6YQz9..."
4,2018-03-20 02:00:00,2018-04-19 03:00:00,"(2018, 27, [UEhsFEgsI5U, yvYeAdcM2Ss, eljAjPH6..."
5,2018-03-20 02:00:00,2018-04-19 03:00:00,"(video, 24, [lq0WUudJiSM, 6zRtrLBg3vk, dl7tsRk..."
6,2018-03-20 02:00:00,2018-04-19 03:00:00,"(humor, 24, [tQR5G3kvfNQ, H4NjcO2GAMU, BYG1qf3..."
7,2018-03-20 02:00:00,2018-04-19 03:00:00,"(official, 23, [92RkVMMvgAU, 6oHn_mmeUOc, Iwz4..."
8,2018-03-20 02:00:00,2018-04-19 03:00:00,"(live, 22, [HfIgfy0HJHU, ofdSsBKW6NU, LAQRtjvt..."
9,2018-03-20 02:00:00,2018-04-19 03:00:00,"(Pop, 22, [HgknAaKNaMM, EROqrFqHegs, h0xHxZpNA..."


In [211]:
# Transform to month's schema
month_df = (
    top_tags_df
        .groupBy("start_date", "end_date")
        .agg(F.collect_list(col("tag_stat")).alias("tags"))
        .select(F.struct(
            col("start_date"),
            col("end_date"),
            col("tags")
        ).alias("month"))
)
df_print(month_df)

,month
0,"(2018-03-20 02:00:00, 2018-04-19 03:00:00, [(c..."
1,"(2017-10-21 03:00:00, 2017-11-20 02:00:00, [(f..."
2,"(2018-01-19 02:00:00, 2018-02-18 02:00:00, [(f..."
3,"(2017-11-20 02:00:00, 2017-12-20 02:00:00, [(f..."
4,"(2017-12-20 02:00:00, 2018-01-19 02:00:00, [(f..."


In [212]:
# Transform to months' schema
months_df = convert_df_to_column_array(month_df, from_col="month", to_col="months")
df_print(months_df)

,months
0,"[(2018-03-20 02:00:00, 2018-04-19 03:00:00, [R..."


## Query 4

**Description:** Show the top 20 channels by the number of views for the whole period.
Note, if there are multiple appearances of the same video for some channel,
you should take into account only the last appearance (with the highest
number of views).

In [213]:
# Create initial dataframe for the query
df = videos_df\
        .withColumn("trending_date", F.to_date(F.from_unixtime(F.unix_timestamp(col("trending_date"), "yy.dd.MM"))))\
        .select(
            col("channel_title"),
            col("video_id"),
            col("trending_date"),
            col("views").cast(t.LongType()).alias("views")
        )


# Group by channel_title and video_id. Take video with max number of views for the same channel
window = w.partitionBy("channel_title", "video_id").orderBy(col("views").desc())
filtered_videos_df = (
      df
        .withColumn("rank", F.row_number().over(window))
        .where(col("rank") == 1)
)
df_print(filtered_videos_df)

,channel_title,video_id,trending_date,views,rank
0,Andrew Lowe,IiRmtBZu0pM,2018-02-12,907719,1
1,Janelle Monáe,uovntV3ZMDc,2018-05-30,2991013,1
2,Klossy,mjBIsBPG1N8,2017-11-28,238028,1
3,Late Night with Seth Meyers,VtZV_rFnI-4,2018-05-14,1669936,1
4,Man City,H6tRAXugvHI,2018-02-20,20443,1


In [214]:
# Group by channel_title to get a number of views for all popular videos.
# Also get top 20 channels by the number of views for the whole period
most_popular_channels = (
    filtered_videos_df
        .withColumn("video_stat", 
                    F.struct(
                       col("video_id"),
                       col("views")
                   ))
        .groupBy("channel_title")
        .agg(
            F.sum(col("views")).alias("total_views"),
            F.min("trending_date").alias("start_date"),
            F.max("trending_date").alias("end_date"),
            F.collect_list(col("video_stat")).alias("videos_views")
        )
        .orderBy(col("total_views").desc())
        .select(
            F.struct(
                col("channel_title").alias("channel_name"),
                col("start_date"),
                col("end_date"),
                col("total_views"),
                col("videos_views")
            ).alias("channel")
        )
        .limit(20)
)
df_print(most_popular_channels, 20)

,channel
0,"(Ozuna, 2017-11-17, 2018-06-02, 487997039, [(C..."
1,"(NickyJamTV, 2018-04-07, 2018-06-14, 465469058..."
2,"(DrakeVEVO, 2018-03-25, 2018-05-13, 364312023,..."
3,"(Flow La Movie, 2018-05-18, 2018-05-18, 337621..."
4,"(Bad Bunny, 2018-03-23, 2018-03-23, 328860380,..."
5,"(Ed Sheeran, 2017-11-27, 2018-06-02, 297553382..."
6,"(ibighit, 2017-11-24, 2018-06-14, 296558059, [..."
7,"(ChildishGambinoVEVO, 2018-06-11, 2018-06-11, ..."
8,"(Marvel Entertainment, 2017-11-29, 2018-06-06,..."
9,"(TaylorSwiftVEVO, 2017-11-16, 2018-04-17, 2307..."


In [215]:
q4_channels_df = convert_df_to_column_array(most_popular_channels, from_col="channel", to_col="channels")
df_print(q4_channels_df)

,channels
0,"[(Ozuna, 2017-11-17, 2018-06-02, 487997039, [R..."


## Query 5

**Description:** Show the top 10 channels with videos trending for the highest number of days
(it doesn't need to be a consecutive period of time) for the whole period.
In order to calculate it, you may use the results from the question No1.
The total_trending_days count will be a sum of the numbers of trending days
for videos from this channel.

In [216]:
# Use detailed_most_trending_df, which was created for query 1
df_print(detailed_most_trending_df, 10)

,video_id,num_trending_days,title,description,trending_day
0,BhIEIO0vaBE,38,To Our Daughter,Directed by Tyler Ross @wttyler\nMusic by Jaco...,"(2018-02-05, 0, 0, 20921796)"
1,NooW_RbfdWI,38,Jurassic World: Fallen Kingdom - Official Trai...,Jurassic World: Fallen Kingdom \nIn Theaters J...,"(2018-02-05, 61833, 1416, 1999326)"
2,2z3EUY1aXdY,38,Justin Timberlake’s FULL Pepsi Super Bowl LII ...,Justin Timberlake breaks it down during the Pe...,"(2018-02-05, 50251, 15239, 2027569)"
3,BhIEIO0vaBE,38,To Our Daughter,Directed by Tyler Ross @wttyler\nMusic by Jaco...,"(2018-02-06, 0, 0, 35832484)"
4,2z3EUY1aXdY,38,Justin Timberlake’s FULL Pepsi Super Bowl LII ...,Justin Timberlake breaks it down during the Pe...,"(2018-02-06, 123302, 39422, 8313413)"
5,NooW_RbfdWI,38,Jurassic World: Fallen Kingdom - Official Trai...,Jurassic World: Fallen Kingdom \nIn Theaters J...,"(2018-02-06, 162435, 4232, 8293323)"
6,BhIEIO0vaBE,38,To Our Daughter,Directed by Tyler Ross @wttyler\nMusic by Jaco...,"(2018-02-07, 0, 0, 40801423)"
7,2z3EUY1aXdY,38,Justin Timberlake’s FULL Pepsi Super Bowl LII ...,Justin Timberlake breaks it down during the Pe...,"(2018-02-07, 143106, 46697, 10226518)"
8,NooW_RbfdWI,38,Jurassic World: Fallen Kingdom - Official Trai...,Jurassic World: Fallen Kingdom \nIn Theaters J...,"(2018-02-07, 200007, 5377, 11883172)"
9,BhIEIO0vaBE,38,To Our Daughter,Directed by Tyler Ross @wttyler\nMusic by Jaco...,"(2018-02-08, 0, 0, 44818108)"


In [217]:
video_id_to_channel_df = videos_df.select(['video_id', 'channel_title']).distinct()

# Get channel names for each video_id
full_detailed_most_trending_df = (
    detailed_most_trending_df.alias('df1')
        .join(video_id_to_channel_df.alias('df2'),
                col("df1.video_id") == col("df2.video_id"), how='inner')
        .select(
            col("channel_title").alias("channel_name"),
            F.struct(
                col("df1.video_id").alias("video_id"),
                col("df1.title").alias("video_title"),
                col("df1.num_trending_days").alias("trending_days")
            ).alias("video_day")
        )
)


# Find top 10 channels with videos trending for the highest number of days
top_channels_df = (
    full_detailed_most_trending_df
        .groupBy("channel_name")
        .agg(
            F.collect_list(col("video_day")).alias("videos_days"),
            F.sum(col("video_day.trending_days")).alias("total_trending_days")
        )
        .orderBy(col("total_trending_days").desc())
        .select(
            F.struct(
                col("channel_name"),
                col("total_trending_days"),
                col("videos_days")
            ).alias("channel")
        ).limit(10)
)
df_print(top_channels_df, 10)

,channel
0,"(Universal Pictures, 1444, [(NooW_RbfdWI, Jura..."
1,"(NFL, 1444, [(2z3EUY1aXdY, Justin Timberlake’s..."
2,"(Anne-Marie, 1444, [(Il-an3K9pjg, Anne-Marie -..."
3,"(MiguelVEVO, 1444, [(u_C4onVrr8U, Miguel - Com..."
4,"(Kylie Jenner, 1444, [(BhIEIO0vaBE, To Our Dau..."
5,"(Sony Pictures Entertainment, 1369, [(dzxFdtWm..."
6,"(Maroon5VEVO, 1369, [(4uTNVumfm84, Maroon 5 - ..."
7,"(FlorenceMachineVEVO, 1369, [(5GHXEGz3PJg, Flo..."
8,"(RoyceDa59VEVO, 1369, [(dARAN1z2KqY, Royce da ..."
9,"(FiftyShadesVEVO, 1369, [(ABLsSWE06q4, Whethan..."


In [218]:
q5_channels_df = convert_df_to_column_array(top_channels_df, from_col="channel", to_col="channels")
df_print(q5_channels_df)

,channels
0,"[(Universal Pictures, 1444, [Row(video_id='Noo..."


## Query 6

**Description:** Show the top 10 videos by the ratio of likes/dislikes for each category
for the whole period. You should consider only videos with more than 100K views.
If the same video occurs multiple times you should take the record when
the ratio was the highest.

In [219]:
# Create initial dataframe for the query. Filter videos, which have more than 100K views
df = (
    videos_df
        .where(col("views") > 100_000)
        .select(
            col('video_id'),
            col("category_id"),
            col("title").alias("video_title"),
            col("views").cast(t.LongType()).alias("views"),
            col("likes").cast(t.LongType()).alias("likes"), 
            col("dislikes").cast(t.LongType()).alias("dislikes")
        )
)


# Count the ratio of likes/dislikes for each video. 
# If the same video occurs multiple times you should take the record when the ratio was the highest
video_ratio_window = w.partitionBy("video_id").orderBy(col("ratio_likes_dislikes").desc())
video_ratios_df = (
       df
        .withColumn("ratio_likes_dislikes", col("likes") / col("dislikes"))
        .withColumn("video_ratio_rank", F.row_number().over(video_ratio_window))
        .where(col("video_ratio_rank") == 1)
)


# Group by category and get top 10 videos by the ratio of likes/dislikes for each category
category_ratio_window =  w.partitionBy("category_id").orderBy(col("ratio_likes_dislikes").desc())
category_ratios_df = (
    video_ratios_df.alias("df1")
        .withColumn("category_ratio_rank", F.row_number().over(category_ratio_window))
        .where(col("category_ratio_rank") <= 10)
        .join(video_categories_df.alias("df2"),
             col("df1.category_id") == col("df2.id"))
        .select(
            col("category_id"),
            col("df2.title").alias("category_name"),
            F.struct(
                col("video_id"),
                col("video_title"),
                col("ratio_likes_dislikes"),
                col("views")
            ).alias("video")
        )
)
df_print(category_ratios_df, 20)

,category_id,category_name,video
0,15,Pets & Animals,"(3jtjkXSvuJw, Try Not to Laugh at This Silly K..."
1,15,Pets & Animals,"(45tSUnisIos, 5 Signs You Work With Cats, 227...."
2,15,Pets & Animals,"(KKwmGuQaMfo, EPIC DIY Gingerbread House for C..."
3,15,Pets & Animals,"(SJsvPv5Q_oY, KITTEN and CAT, 171.971830985915..."
4,15,Pets & Animals,"(_4_V7ZGCYDE, Cat with Short Legs Looks Like a..."
5,15,Pets & Animals,"(rplCCSqWXIY, Head Over Heels (A Valentines Sp..."
6,15,Pets & Animals,"(OPtomB--3Ss, What Cat People Do When It's Col..."
7,15,Pets & Animals,"(fS2fsa1fuIw, If It Fits, I Sits..., 106.71014..."
8,15,Pets & Animals,"(DRfniGCgW0I, Cat Alarm Clock, 96.478723404255..."
9,15,Pets & Animals,"(x8DegXNFOns, What's Your Cat's Petting Style?..."


In [220]:
top_category_videos_df = (
    category_ratios_df
        .groupBy("category_id", "category_name")
        .agg(
            F.collect_list(col("video")).alias("videos")
        )
        .select(
            F.struct(
                col("category_id"),
                col("category_name"),
                col("videos")
            ).alias("category")
        )
)
df_print(top_category_videos_df, 10)

,category
0,"(15, Pets & Animals, [(3jtjkXSvuJw, Try Not to..."
1,"(22, People & Blogs, [(DrApVH3afMI, some bloop..."
2,"(28, Science & Technology, [(UCKC-QVcVn0, The ..."
3,"(27, Education, [(WrcwcMIYpQg, FLOWER BOY: a c..."
4,"(17, Sports, [(Vm4Xn5vjqRA, CRISTIANO RONALDO ..."
5,"(26, Howto & Style, [(o8lSb-UOA3M, Dan and Phi..."
6,"(19, Travel & Events, [(-trTIbE3N4Y, I LOVE JA..."
7,"(23, Comedy, [(LVPzYHk5VW8, Daniel Caesar and ..."
8,"(25, News & Politics, [(QfuOtGLfg04, What is H..."
9,"(24, Entertainment, [(nc6pLSQGcHQ, Shawn Mende..."


In [221]:
q6_category_videos_df = convert_df_to_column_array(top_category_videos_df, from_col="category", to_col="categories")
df_print(q6_category_videos_df)

,categories
0,"[(15, Pets & Animals, [Row(video_id='3jtjkXSvu..."
